In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
import datetime
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('train.csv', parse_dates=['fecha'])
df_test = pd.read_csv('test.csv')

In [3]:
def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0):
        return metroscubiertos
    else:
        return metrostotales

In [4]:
dolar= pd.read_csv('Datos históricos USD_MXN.csv')
dolar.columns = ["fecha", "last", "opening", "max", "min", "std"]
dolar.drop(["std", "opening", "opening", "max", "min"], axis=1 ,inplace=True)
#Paso a formato fecha correctamente y luego creo columnas mes y año
dolar['date'] = pd.to_datetime(dolar['fecha'], format="%b %Y")
dolar.drop(["fecha"], axis=1, inplace=True)
dolar.columns= ['price', "date"]
#El csv ponia los numeros con punto en vez de coma.
dolar['price'] = dolar['price'].replace(',', '.', regex=True).astype(float)
dolar["year"] = dolar.date.dt.year
dolar['month'] = dolar.date.dt.month
dolar.drop(["date"], axis=1, inplace=True)

In [5]:
def setDataFrameWithDollarPrice(dfReceived, dolar):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month

    #Agrego la columna precio en dolar por propiedad, en el dataframe recibido.
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    dfReceived["dollar_price"] = dfReceived.apply(lambda row: row["precio"]/row["price"], axis=1)
    #Filtro dejando las cosas útiles.
    #Chequeado de no dejar alguna importante afuera
    dfReceived = dfReceived.filter(["id", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha',
                   'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                   'centroscomercialescercanos', 'price','dollar_price'])
    print(dfReceived.columns)


    
    #Limpiando las propiedades tipo casa, no pueden no tener al menos una habitacion o baño y no pueden
    #tener metros no cubiertos
    filtro1 = dfReceived["tipodepropiedad"].isin(["Apartamento", "Casa", "Casa en condominio",
                                        "Casa uso de suelo", "Rancho", "Quinta Vacacional"])
    dfCasas = dfReceived[filtro1]
    dfCasas = dfCasas[np.isfinite(dfCasas['habitaciones'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['banos'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['metroscubiertos'])]
    dfCasas.garages.fillna(0, inplace=True)

    dfReceived = pd.merge(dfCasas, dfReceived, on=["id", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                  'centroscomercialescercanos', 'price', 'dollar_price'], how='inner')
    #dfReceived = pd.merge(dfCasas, df, on=["id"], how = 'inner')
    dfReceived.rename(columns={"dollar_price": "precio"}, inplace = True)
    dfReceived.rename(columns={"price": "precioDelDolar"}, inplace = True)
    return dfReceived

In [6]:
df_train = setDataFrameWithDollarPrice(df, dolar)
df_train

Index(['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'ciudad',
       'provincia', 'metroscubiertos', 'metrostotales', 'idzona', 'fecha',
       'year', 'month', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'antiguedad', 'centroscomercialescercanos', 'price',
       'dollar_price'],
      dtype='object')


,id,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,idzona,...,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio
0,254099,Apartamento,2.0,1.0,2.0,Benito Juárez,Distrito Federal,80.0,80.0,23533.0,...,2015,8,0.0,0.0,0.0,0.0,NaN,0.0,16.7522,135683.671398
1,53461,Casa en condominio,3.0,2.0,2.0,La Magdalena Contreras,Distrito Federal,268.0,180.0,24514.0,...,2013,6,0.0,0.0,0.0,1.0,10.0,1.0,12.9460,278078.170864
2,247984,Casa,3.0,2.0,2.0,Tonalá,Jalisco,144.0,166.0,48551.0,...,2015,10,0.0,0.0,0.0,0.0,5.0,0.0,16.5000,72727.272727
3,209067,Casa,2.0,1.0,1.0,Zinacantepec,Edo. de México,63.0,67.0,53666.0,...,2012,3,0.0,0.0,0.0,1.0,1.0,1.0,12.8070,50753.494183
4,185997,Apartamento,2.0,1.0,1.0,Zapopan,Jalisco,95.0,95.0,47835.0,...,2016,6,0.0,0.0,0.0,0.0,10.0,0.0,18.2800,62910.284464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174992,119879,Casa,2.0,2.0,1.0,Zinacantepec,Edo. de México,67.0,NaN,53666.0,...,2015,2,0.0,0.0,0.0,0.0,0.0,0.0,14.9507,43476.225193
174993,259178,Casa,3.0,3.0,3.0,Toluca,Edo. de México,200.0,250.0,51954.0,...,2014,7,0.0,0.0,0.0,1.0,0.0,1.0,13.2177,146772.887870
174994,131932,Apartamento,2.0,1.0,2.0,Benito Juárez,Distrito Federal,138.0,138.0,50003995.0,...,2015,3,0.0,0.0,0.0,0.0,20.0,0.0,15.2618,222778.440289
174995,146867,Casa,4.0,0.0,4.0,Iztapalapa,Distrito Federal,235.0,137.0,24162.0,...,2014,12,1.0,0.0,0.0,1.0,20.0,1.0,14.7490,195945.487830


In [7]:
def agregarAnioMesYPrecioDelDolarALaFecha(dfReceived):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    return dfReceived

In [8]:
# Droppeamos strings y columnas complejas y repetidas
drop_cols_train = ['fecha', 'idzona']
drop_cols_test = ['titulo', 'descripcion', 'direccion', 'lat', 'lng', 'fecha', 'idzona']
df_train = df_train.drop(drop_cols_train, axis=1).copy()

df_test = agregarAnioMesYPrecioDelDolarALaFecha(df_test)
df_test = df_test.drop(drop_cols_test, axis=1).copy()
print(f"Columnas ({len(df_train.columns)}): {df_train.columns.tolist()}")
df_train.head()

Columnas (19): ['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'ciudad', 'provincia', 'metroscubiertos', 'metrostotales', 'year', 'month', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad', 'centroscomercialescercanos', 'precioDelDolar', 'precio']


,id,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio
0,254099,Apartamento,2.0,1.0,2.0,Benito Juárez,Distrito Federal,80.0,80.0,2015,8,0.0,0.0,0.0,0.0,NaN,0.0,16.7522,135683.671398
1,53461,Casa en condominio,3.0,2.0,2.0,La Magdalena Contreras,Distrito Federal,268.0,180.0,2013,6,0.0,0.0,0.0,1.0,10.0,1.0,12.9460,278078.170864
2,247984,Casa,3.0,2.0,2.0,Tonalá,Jalisco,144.0,166.0,2015,10,0.0,0.0,0.0,0.0,5.0,0.0,16.5000,72727.272727
3,209067,Casa,2.0,1.0,1.0,Zinacantepec,Edo. de México,63.0,67.0,2012,3,0.0,0.0,0.0,1.0,1.0,1.0,12.8070,50753.494183
4,185997,Apartamento,2.0,1.0,1.0,Zapopan,Jalisco,95.0,95.0,2016,6,0.0,0.0,0.0,0.0,10.0,0.0,18.2800,62910.284464


In [9]:
display(df_train.isnull().sum())
numeric_columns_with_nulls = list(set(df_train.columns[df_train.isnull().sum() > 0].tolist()) - 
                                  set(['tipodepropiedad', 'ciudad', 'provincia']))
print(numeric_columns_with_nulls)

id                                0
tipodepropiedad                   0
habitaciones                      0
garages                           0
banos                             0
ciudad                          178
provincia                        48
metroscubiertos                   0
metrostotales                 36696
year                              0
month                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
antiguedad                    24006
centroscomercialescercanos        0
precioDelDolar                    0
precio                            0
dtype: int64

['metrostotales', 'antiguedad']


In [10]:
# Para los nulls numéricos, usar un Imputer con strategy mean (reemplazamos los NaN por el promedio)
# Para no leakear, spliteamos el dataset antes
df_train_ohe = pd.get_dummies(df_train, dummy_na=True)
print("First")
df_test_ohe = pd.get_dummies(df_test, drop_first=True)

First


In [11]:
X = df_train_ohe.drop("precio", axis=1)
y = df_train_ohe['precio']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

#display(df_train_ohe)

In [12]:
#----------------PREPROCESAMIENTO----------------

#df_train_xgboost = df_train_xgboost.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
#df_test_xgboost = df_test_xgboost.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
#df_train_xgboost = df_train_xgboost.fillna(value = {'provincia': 0, 'tipodepropiedad': 0})

#print(df_train_xgboost.shape)
#XGBoost maneja los NaNs, ver despues si lo dejamos como esta

#df_train_xgboost_ohe = pd.get_dummies(df_train_xgboost, drop_first=True, dummy_na = True)#, sparse = True)
#df_test_xgboost_ohe = pd.get_dummies(df_test_xgboost, drop_first=True)#, sparse = True)

#print(df_train_xgboost.shape)
#precio = df_train_xgboost['precio']
#df_train_xgboost.drop(labels=['precio'], axis=1,inplace = True)
#df_train_xgboost.insert(len(df_train_xgboost.columns), 'Precio', precio)

#X_train, y_train = df_train_xgboost.iloc[:,:-1],df_train_xgboost.iloc[:,-1]

#X = df_train_xgboost_ohe.drop("precio", axis=1)
#y = df_train_xgboost_ohe['precio']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

#df_test_xgboost['tipodepropiedad_Garage'] = 0
#df_test_xgboost['tipodepropiedad_Hospedaje'] = 0
#df_test_xgboost = df_test_xgboost[list(X_train.columns.values)]

xgb1 = XGBRegressor()

#Ir cambiando parametros para ver como functiona

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.04], #probamos con 0.03,0.04,0.05,0.06,0.07
              'max_depth': [9], #probamos con 5,6,7,8,9,10,11
              'min_child_weight': [1], #Probamos con 1,2,3,4,5,6
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7], #probamos 0.3,0.4,0.5,0.6,0.7,0.8
              'n_estimators': [700],  #Se probo con 400,500,600,700,800
            'gamma': [0]} #Se probo con 0,1,5,10

#Estos son sin correccion de metros
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7214816192533363
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7218556040564428
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7219040830119198
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7242695855881088
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 2, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7245128843441556
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7246402143520269

#Con fechas nos da 0.7438760200186147
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X,
         y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s


MemoryError: Unable to allocate array with shape (87498, 731) and data type float64

In [2]:
#df_ciudades = df_train.groupby('ciudad').agg({'metrostotales':'mean'})

#df_ciudades.reset_index(inplace = True)
#ciudades_list = df_ciudades.values.tolist()
#ciudades_list.append(['Ciudad del Maíz',0])

#print(ciudades_list)
        
#for i in range (0,len(ciudades_list)):
#    df_train.loc[df_train.ciudad == ciudades_list[i][0], 'ciudad'] = float(ciudades_list[i][1])
#    df_test.loc[df_test.ciudad == ciudades_list[i][0], 'ciudad'] = float(ciudades_list[i][1])
    
#df_train.apply(lambda col:pd.to_numeric(col, errors='coerce'))
#df_test.apply(lambda col:pd.to_numeric(col, errors='coerce'))
#df_train.astype({'ciudad': 'float64'})
#df_test.astype({'ciudad': 'float64'})

#df_train.head(10)

#print(df_train.shape)
#print(df_test.shape)
#print(df_train.dtypes)

[['Abalá', 200.0], ['Abasolo', 287.8333333333333], ['Abejones', nan], ['Acajete', 196.33333333333334], ['Acambay', 186.6], ['Acaponeta', 380.0], ['Acapulco de Juárez', 168.98036135113904], ['Acateno', nan], ['Acatic', nan], ['Acatzingo', 211.0], ['Acayucan', 68.0], ['Acolman', 108.77777777777777], ['Aconchi', 250.0], ['Actopan', 138.375], ['Acuitzio', nan], ['Aculco', 335.5], ['Acuña', 153.66666666666666], ['Agua Dulce', 142.0], ['Agua Prieta', 208.5], ['Aguascalientes', 176.5063957863055], ['Ahome', 190.90076335877862], ['Ahuacatlán', nan], ['Ahualulco', nan], ['Ahualulco de Mercado', 170.0], ['Ahuazotepec', 400.0], ['Ajacuba', nan], ['Alamo Temapache', 99.0], ['Alamos', nan], ['Aldama', 250.57142857142858], ['Allende', 204.4], ['Almoloya', nan], ['Almoloya de Alquisiras', nan], ['Almoloya de Juárez', 94.0], ['Almoloya del Río', nan], ['Altamira', 174.93867924528303], ['Altotonga', 147.0], ['Alvarado', 203.52032520325204], ['Alvaro Obregón', 188.41555023923445], ['Amacuzac', 227.0], [

ValueError: could not convert string to float: 'Molango de Escamilla'

In [3]:
df_train_xgboost = df_train
df_test_xgboost = df_test

print(df_train_xgboost.head())

def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0):
        return metroscubiertos
    else:
        return metrostotales
    
def asignarMetrosLibres(metroscubiertos, metrostotales):
    if (metroscubiertos != metrostotales):
        return metrostotales-metroscubiertos
    else:
        return 0

df_train_xgboost['fecha'] = pd.to_datetime(df_train_xgboost['fecha'])
df_train_xgboost['year'] = df_train_xgboost['fecha'].dt.year
df_train_xgboost['month']= df_train_xgboost['fecha'].dt.month
df_train_xgboost['day']= df_train_xgboost['fecha'].dt.day
df_train_xgboost.drop(["fecha"], axis=1, inplace=True)

df_test_xgboost['fecha'] = pd.to_datetime(df_test_xgboost['fecha'])
df_test_xgboost['year'] = df_test_xgboost['fecha'].dt.year
df_test_xgboost['month']= df_test_xgboost['fecha'].dt.month
df_test_xgboost['day']= df_test_xgboost['fecha'].dt.day
df_test_xgboost.drop(["fecha"], axis=1, inplace=True)
    
#df_train_xgboost['metrostotales'].fillna(0, inplace=True)
#df_train_xgboost['metrostotales'] = df_train_xgboost.apply(lambda x: asignarMetros(x['metroscubiertos'],x['metrostotales']),axis=1)
#df_train_xgboost['metroslibres'] = df_train_xgboost.apply(lambda x: asignarMetrosLibres(x['metroscubiertos'],x['metrostotales']),axis=1)

#df_test_xgboost['metrostotales'].fillna(0, inplace=True)
#df_test_xgboost['metrostotales'] = df_test_xgboost.apply(lambda x: asignarMetros(x['metroscubiertos'],x['metrostotales']),axis=1)
#df_test_xgboost['metroslibres'] = df_test_xgboost.apply(lambda x: asignarMetrosLibres(x['metroscubiertos'],x['metrostotales']),axis=1)

       id                                             titulo  \
0  254099                                  depto. tipo a-402   
1   53461                     condominio horizontal en venta   
2  247984              casa en venta urbi 3 recamaras tonala   
3  209067  casa sola en toluca zinacantepec con credito i...   
4  185997                                     paseos del sol   

                                         descripcion     tipodepropiedad  \
0  depto. interior de 80.15m2, consta de sala com...         Apartamento   
1  <p>entre sonora y guerrero, atr&aacute;s del h...  Casa en condominio   
2  descripcion \nla mejor ubicacion residencial e...                Casa   
3  casa en privada con caseta de vigilancia casas...                Casa   
4  bonito departamento en excelentes condiciones ...         Apartamento   

                         direccion   ciudad         provincia  antiguedad  \
0  Avenida Division del Norte 2005  138.117  Distrito Federal         NaN   
1   